# Guía de Estudio: Redes Neuronales Recurrentes (RNN) con TensorFlow y Keras
**Autora: Ximena Carolina Fernandez Cardenas**

## 1. Introducción a las Redes Neuronales Recurrentes (RNN)

Las **Redes Neuronales Recurrentes (RNN)** son un tipo de red diseñada para trabajar con datos secuenciales, como texto, series temporales, audio, etc. Su característica principal es que pueden **mantener memoria del estado anterior** mediante conexiones recurrentes.

## 2. Características clave

- Manejan secuencias de longitud variable.
- Mantienen un estado interno que captura dependencias temporales.
- Pueden usarse para tareas de clasificación, predicción de secuencia, generación de texto, etc.

## 3. Arquitecturas relacionadas

- **Simple RNN**: estructura básica, propensa al desvanecimiento del gradiente.
- **LSTM**: memoria a largo plazo, evita el problema de gradientes.
- **GRU**: similar a LSTM pero con una arquitectura más simple.

In [5]:
# Paso 1: Importar librerías
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [18]:
# Paso 1: Cargar y preprocesar el dataset (Shakespeare)
dataset, info = tfds.load("tiny_shakespeare", with_info=True)

text_data = ""
for example in tfds.as_numpy(dataset['train']):
    # 'example' es un diccionario con clave 'text'
    text_data += example['text'].decode('utf-8')

# Crear vocabulario y mapeos
vocab = sorted(set(text_data))
char2idx = {char: i for i, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text_data])

In [19]:
# Paso 2: Preparar el conjunto de datos para entrenamiento
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Parámetros de entrenamiento
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [22]:
# Paso 3: Construir el modelo RNN
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
        ),
        tf.keras.layers.SimpleRNN(
            units=rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform'
        ),
        tf.keras.layers.Dense(vocab_size)
    ])

model = build_model(vocab_size, embedding_dim, rnn_units)



In [23]:
# Paso 4: Función de pérdida y compilación
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [ ]:
# Paso 5: Entrenamiento
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 330s 2s/step - loss: 3.0610
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - loss: 2.0898
Epoch 3/10
122/155 ━━━━━━━━━━━━━━━━━━━━ 1:09 2s/step - loss: 1.8614

In [ ]:
# Paso 6: Generación de texto
def generate_text(model, start_string, num_generate=100):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

print("\n--- Texto generado ---\n")
print(generate_text(model, start_string="To be, or not to be: "))

## 6. Explicación de funciones y parámetros

### `Embedding`:
Capa que convierte cada carácter (índice entero) en un vector denso.

- `input_dim=vocab_size`: tamaño del vocabulario.
- `output_dim=embedding_dim`: dimensión del vector de salida.

### `SimpleRNN`:
- `units=rnn_units`: número de neuronas en la capa RNN.
- `return_sequences=True`: devuelve la secuencia completa (necesario para múltiples capas o salida secuencial).
- `stateful=True`: mantiene estado entre lotes, útil para secuencias largas o texto continuo.

### `Dense`:
Capa de salida que proyecta hacia `vocab_size` (una predicción por carácter).

---

### `loss`:
Se utiliza entropía cruzada categórica con `from_logits=True` porque la salida de la red **no pasa por softmax** (la capa `Dense` genera logits crudos).

---

## Conclusiones

- Las **RNN** son poderosas para trabajar con secuencias, pero tienen limitaciones (como el desvanecimiento del gradiente) que las arquitecturas **LSTM** y **GRU** resuelven mejor.
- **Keras** permite construir RNNs con pocas líneas de código.
- Este pipeline es útil para **generación de texto**, **análisis de sentimientos**, predicción de series temporales, entre otros casos.
